In [1]:
import os
import pickle
from scipy import sparse
os.listdir('../data')

['dev.csv',
 'dev_binary_full_data.pckl',
 'dev_binary_vectorized.pckl',
 'dev_count_full_data.pckl',
 'dev_count_vectorized.pckl',
 'dev_hashing_binary_full_data.pckl',
 'dev_hashing_binary_vectorized.pckl',
 'dev_hashing_full_data.pckl',
 'dev_hashing_vectorized.pckl',
 'dev_labels.pckl',
 'dev_num_caps.pckl',
 'dev_num_exclam.pckl',
 'dev_ratings.pckl',
 'dev_reviewsToDate.pckl',
 'dev_tfidf_full_data.pckl',
 'dev_tfidf_vectorized.pckl',
 'should_be_empty.txt',
 'train.csv',
 'train_binary_full_data.pckl',
 'train_binary_vectorized.pckl',
 'train_count_full_data.pckl',
 'train_count_vectorized.pckl',
 'train_hashing_binary_full_data.pckl',
 'train_hashing_binary_vectorized.pckl',
 'train_hashing_full_data.pckl',
 'train_hashing_vectorized.pckl',
 'train_labels.pckl',
 'train_num_caps.pckl',
 'train_num_exclam.pckl',
 'train_ratings.pckl',
 'train_reviewsToDate.pckl',
 'train_tfidf_full_data.pckl',
 'train_tfidf_vectorized.pckl']

In [2]:
vectorizers = ['count', 'tfidf', 'hashing', 'binary', 'hashing_binary']

for vectorizer in vectorizers:
    for dataset in ['train', 'dev']:
        with open(f'../data/{dataset}_{vectorizer}_vectorized.pckl', 'rb') as f:
            vectorized_data = pickle.load(f)
        
        with open(f'../data/{dataset}_num_caps.pckl', 'rb') as f:
            caps = pickle.load(f)
        
        with open(f'../data/{dataset}_num_exclam.pckl', 'rb') as f:
            exclam = pickle.load(f)
            
        with open(f'../data/{dataset}_reviewsToDate.pckl', 'rb') as f:
            rev_counts = pickle.load(f)
            
        with open(f'../data/{dataset}_ratings.pckl', 'rb') as f:
            ratings = pickle.load(f)
        
        caps = caps.values.reshape(-1, 1)
        exclam = exclam.values.reshape(-1, 1)
        rev_counts = rev_counts.values.reshape(-1, 1)
        ratings = ratings.values.reshape(-1, 1)
        
        # more features
        
        full_data = sparse.hstack((vectorized_data,
                                   sparse.csr_matrix(caps),
                                   sparse.csr_matrix(exclam),
                                   sparse.csr_matrix(rev_counts),
                                   sparse.csr_matrix(ratings),
                                   # more features
                                   ))
        
        with open(f'../data/{dataset}_{vectorizer}_full_data.pckl', 'wb') as f:
            pickle.dump(full_data, f)